## SUPPOSED TO COME AFTER INITIALIZING DATA and FUNCTIONS IN THE draft-analysis notebook

moved here for organizational purposes

In [ ]:
for file in os.listdir('./XYZ_Files/'):
    if "Theta" in file:
        name=file.split('.')[0]
        lammps_file = "UNK_A7CF9B.lmp"
        output = f"{name}.lmp"
        new_coords = read_xyz(f"./XYZ_Files/{file}")
        update_lammps_data(lammps_file, new_coords, output)

In [ ]:
%%time
import os


template_input = '''units           real
atom_style      full
pair_style      lj/cut/coul/cut 10.0 13.4
bond_style      harmonic
angle_style     harmonic
dihedral_style  opls
special_bonds   lj/coul 0 0 0.5
improper_style  cvff
pair_modify mix geometric
dielectric 4.9
read_data       {data_file}
timestep 1.0
thermo 1
thermo_style multi
neighbor 2.0 bin
run_style verlet
run 0
'''

files = os.listdir('./lammps_files_ligpargen/')
lmp_files = [file for file in files if file.endswith(".lmp")]

for file in tqdm(lmp_files):
    if file.endswith(".lmp"):
        name=file.split(".")[0]
        data_file_path = os.path.join('./lammps_files_ligpargen/', file)
        input_file_path = os.path.join('./lammps_files_ligpargen/', f"{name}.lmpin")
        
        # Create an input file for LAMMPS, replacing ${data_file} with the actual file name
        with open(input_file_path, 'w') as f:
            f.write(template_input.format(data_file=data_file_path))

        # Run LAMMPS using subprocess
        # print(f"running {name}")
        # print(" ".join(["/usr/bin/lmp", "-in", input_file_path, "-log", f"{name}.log"]))
        with open(os.devnull, 'w') as fnull:
            subprocess.run(["/usr/bin/lmp", "-in", input_file_path, "-log", f"{name}.log"], stdout=fnull)


In [ ]:

# Regular expression for extracting phi and theta from filename
filename_pattern = re.compile(r"_Phi_(\d+)_Theta_(\d+)_")

# Initialize an empty DataFrame to hold all the data
all_data = pd.DataFrame()


files = os.listdir('./lammps_files_ligpargen/')
log_files = [file for file in files if file.endswith(".log")]

# Loop through log files
for filename in log_files:  # Replace with your log files directory
    match = filename_pattern.search(filename)
    if match:
        phi = int(match.group(1))
        theta = int(match.group(2))
        # Find the section with thermo data
        df = parse_lammps_log(os.path.join('./lammps_files_ligpargen/', filename))[0]
    
        # Add theta and phi columns (Assume you have a way to get these from the filename or file content)
        df['theta'] = theta  # Replace with actual value
        df['phi'] = phi  # Replace with actual value
    
        # Append to the overall DataFrame
        all_data = pd.concat([all_data, df], ignore_index=True)

# Convert to appropriate data types
all_data = all_data.apply(pd.to_numeric, errors='ignore')

# Sort by theta and phi
sorted_data = all_data.sort_values(['theta', 'phi'])

In [ ]:
sorted_data

In [ ]:
for phi in set(sorted_data['phi']):
    subset = sorted_data[sorted_data['phi'] == phi]
    # plt.plot(subset['theta'], subset['PotEng'])  # Replace 'E_total' with the actual column name for total energy
    plt.plot(subset['theta'], subset['E_dihed'])  # Replace 'E_total' with the actual column name for total energy
    # plt.plot(subset['theta'], subset['E_dihed'], c="red")  # Replace 'E_total' with the actual column name for total energy
    # plt.plot(subset['theta'], subset['E_impro'], c="black")  # Replace 'E_total' with the actual column name for total energy
plt.xlabel('Theta (degrees)')
plt.ylabel('Total Energy (kcal/mol)')
plt.title(f'Energy components vs Theta for all Phi')
plt.show()

In [ ]:
for phi in set(sorted_data['phi']):
    subset = sorted_data[sorted_data['phi'] == phi]
    # plt.plot(subset['theta'], subset['PotEng'])  # Replace 'E_total' with the actual column name for total energy
    plt.plot(subset['theta'], subset['E_vdwl'])  # Replace 'E_total' with the actual column name for total energy
    # plt.plot(subset['theta'], subset['E_dihed'], c="red")  # Replace 'E_total' with the actual column name for total energy
    # plt.plot(subset['theta'], subset['E_impro'], c="black")  # Replace 'E_total' with the actual column name for total energy
plt.xlabel('Theta (degrees)')
plt.ylabel('Total Energy (kcal/mol)')
plt.title(f'Energy components vs Theta for all Phi')
plt.show()